In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from PIL import Image

In [ ]:
class AnimalsDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.transform = transform
        self.dataset = dataset
    
    def __len__(self):
        return len(self.dataset[0])
    
    def __getitem__(self, idx):
        return (self.dataset[0][idx], self.dataset[1][idx])

In [ ]:
# PyTorch tutorial
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 64)
        self.fc2 = nn.Linear(64, 16)
        self.fc3 = nn.Linear(16, 3)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# Rosebrock
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Conv2d(3, 32, 3, 1, 1)
        self.fc = nn.Linear(32 * 32 * 32, 3)
        
    def forward(self, x):
        x = F.relu(self.conv(x))
        x = x.view(-1, 32 * 32 * 32)
        x = F.softmax(self.fc(x), dim=1)
        return x

In [ ]:
def read_image(root_dir):
    data, labels = [], []
    classes = os.listdir(root_dir)
    paths = [os.path.join(root_dir, c) for c in classes]
    for path in paths:
        files = os.listdir(path)
        filenames = [os.path.join(path, file) for file in files]
        for filename in filenames:
            img = cv2.imread(filename)
            image = cv2.resize(img, (32, 32), interpolation=cv2.INTER_AREA)
            # for PyTorch
            image = np.transpose(image, (2, 0, 1))
            data.append(image.astype('float') / 255.0)
            # labels.append(files[0].split('_')[0])
            labels.append(classes.index(files[0].split('_')[0]))
    data = np.array(data)
    labels = np.array(labels)
    X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size=0.32, random_state=42)
    # Y_train = LabelBinarizer().fit_transform(y_train).argmax(axis=1)
    # Y_test = LabelBinarizer().fit_transform(y_test).argmax(axis=1)
    trainset = (X_train, Y_train)
    testset = (X_test, Y_test)
    return (trainset, testset, classes)

In [ ]:
def CNN(root_dir):
    # Load image and split data into trainset and testset
    trainset, testset, classes = read_image(root_dir)
    trainset, testset = AnimalsDataset(trainset), AnimalsDataset(testset)
    trainloader = DataLoader(trainset, batch_size=4, shuffle=False, num_workers=2)
    testloader = DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)
    
    # Instantiate CNN, pick loss function and optimizer
    net = Net().float()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.95) # lr: 1e-1, 1e-2, 1e-3, 1e-4; momentum: 0.9-0.99

    # Training in 50 epochs
    epochs = 50
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            # print(type(inputs), type(labels), inputs.shape)
            optimizer.zero_grad()
            outputs = net(inputs.float())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print('[info] epoch {:2d}, loss: {:8.3f}'.format(epoch + 1, running_loss))
    print('Finished training.')
    PATH = './animals.pth'
    torch.save(net.state_dict(), PATH)
    print('Saving trained model to {:s}'.format(PATH))

    # Calculate the accuracy
    class_correct = [0. for i in range(3)]
    class_total = [0. for i in range(3)]
    acc = [0. for i in range(3)]
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images.float())
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(3):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    for i in range(3):
        acc[i] = 100 * class_correct[i] / class_total[i]
    
    # Generate classification report
    with torch.no_grad():
        images, labels = list(zip(*testset))
        outputs = net(torch.Tensor(images).float())
        _, predicted = torch.max(outputs, 1)
        report = classification_report(labels, predicted, target_names=classes)
    
    return (trainset, testset, trainloader, testloader, classes, acc, report)

In [ ]:
root_dir = '/Users/libao/Documents/data/animals/'
trainset, testset, trainloader, testloader, classes, accuracies, report = CNN(root_dir)

for i, acc in enumerate(accuracies):
    print('Accuracy of {:5s} : {:.2f} %'.format(classes[i], acc))
print('Classification report:\n', report)

In [ ]:
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(trainloader)
images, labels = dataiter.next()
print(labels.shape)
print(labels)
print(images[0].shape)    
imshow(torchvision.utils.make_grid(images))
print(images[0].dtype)
print(' '.join('{:5s}'.format(classes[labels[j]]) for j in range(4)))

In [ ]:
PATH = './animals.pth'
torch.save(net.state_dict(), PATH)
dataiter = iter(testloader)
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('{:5s}'.format(classes[labels[j]]) for j in range(4)))

net = Net().float()
net.load_state_dict(torch.load(PATH))
outputs = net(images.float())
_, predicted = torch.max(outputs, 1)
print('Predicted: ', ' '.join('{:5s}'.format(classes[predicted[j]]) for j in range(4)))

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images.float())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the test images: {:f} %'.format(100 * correct / total))

In [ ]:
# alexnet = torchvision.models.alexnet(pretrained=True)
# vgg16 = torchvision.models.vgg16(pretrained=True)
# resnet101 = torchvision.models.resnet101(pretrained=True)